# Perfilamiento del DataSet

In [1]:
import re
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [2]:
agua = pd.read_csv('../data/consumo-agua.csv')

In [16]:
agua.sample(2)

,Geo Point,Geo Shape,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des
6960,"19.4885632293,-99.2031811727","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",250.13,2019,Azcapotzalco,33.803735,2805.71,AZCAPOTZALCO,PUEBLO SAN JUAN TLIHUACA,50.026,3059.50,34.376404,3.660000,2,3.66,33963,BAJO
31107,"19.3508617598,-99.2206588357","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",103.70,2019,Álvaro Obregón,47.987273,1583.58,ALVARO OBREGON,AMPLIACION LAS AGUILAS,51.850,1745.23,45.927105,19.316667,3,57.95,59152,BAJO


In [3]:
def cuenta_nulos_por_columnas(df):
    """
    Función que realiza una tabla con la cuenta de missing values por columna y obtiene la proporción que estos missing
    values representan del total.
    """
    valores_nulos = df.isnull().sum()
    porcentaje_valores_nulos = 100 * df.isnull().sum() / len(df)
    tabla_valores_nulos = pd.concat([valores_nulos, porcentaje_valores_nulos], axis=1)
    tabla_valores_nulos_ordenada = tabla_valores_nulos.rename(
        columns={0: 'Missing Values', 1: '% del Total'})
    tabla_valores_nulos_ordenada = tabla_valores_nulos_ordenada[
        tabla_valores_nulos_ordenada.iloc[:, 1] != 0].sort_values(
        '% del Total', ascending=False).round(1)
    print("El dataframe tiene " + str(df.shape[1]) + " columnas.\n"
                                                     "Hay " + str(tabla_valores_nulos_ordenada.shape[0]) +
          " columnas que tienen NA's.")
    return tabla_valores_nulos_ordenada

In [5]:
cuenta_nulos_por_columnas(agua)['Missing Values'].sum()

El dataframe tiene 17 columnas.
Hay 5 columnas que tienen NA's.


26318

## Profiling del data set completo

In [37]:
def genera_profiling_general(df):
    cuenta_de_variables = len(df.columns)
    cuenta_observaciones = len(df)
    total_celdas = cuenta_de_variables*cuenta_observaciones

    # Contamos el tipo de datos del dataset
    vars_type = df.dtypes
    vars_type = pd.DataFrame(vars_type, columns = ['tipo'])

    # Asignamos un valor para cada tipo

    ## Numéricas
    cantidad_numericas = len(vars_type.loc[vars_type["tipo"] == "int64"])
    cantidad_numericas = cantidad_numericas + len(vars_type.loc[vars_type["tipo"] == "float64"])
    #print(cantidad_numericas)

    ## Fechas
    cantidad_fecha = len(vars_type.loc[vars_type["tipo"] == "Date"])
    #print(cantidad_fecha)

    ## Categoricas
    cantidad_categoricas = len(vars_type.loc[vars_type["tipo"] == "category"])
    #print(cantidad_categoricas)

    ## Texto
    cantidad_texto = len(vars_type.loc[vars_type["tipo"] == "object"])
    #print(cantidad_texto)

    # Contamos los faltantes
    nulos_totales = cuenta_nulos_por_columnas(df)['Missing Values'].sum()
    #print(nulos_totales)

    # Obtenemos el porcentaje de datos que son faltantes
    nulos_porcentaje = ((nulos_totales/(total_celdas))*100).round(1).astype(str)+'%'
    #print(nulos_porcentaje)

    # Obtenemos el total de columnas duplicadas
    ds_duplicados = df.duplicated(subset=None, keep='first')
    ds_duplicados = pd.DataFrame(ds_duplicados,columns = ['duplicated'])
    numero_de_duplicados = len(ds_duplicados.loc[ds_duplicados["duplicated"] == True])
    #print(numero_de_duplicados)

    # Obtenemos el porcentaje de duplicados
    porcentaje_de_duplicados = str(((numero_de_duplicados/(total_celdas))*100))+'%'
    #print(porcentaje_de_duplicados)

    estadisticas = ['Total de variables','Conteo de observaciones','Total de celdas',
                        'Cantidad de variables numericas','Cantidad de variables de fecha',
                        'Cantidad de variables categóricas', 'Cantidad de variables de texto',
                        'Valores faltantes','Porcentaje de valores faltantes',
                        'Renglones duplicados', 'Porcentaje de valores duplicados']

    valores_estadisticas = [cuenta_de_variables,cuenta_observaciones,total_celdas,cantidad_numericas,
                        cantidad_fecha,cantidad_categoricas,cantidad_texto,nulos_totales,nulos_porcentaje,
                        numero_de_duplicados,porcentaje_de_duplicados]

    valores = {'Estadisticas':estadisticas,'Resultado':valores_estadisticas}

    df_perfilamiento_general = pd.DataFrame(data=valores)
    return df_perfilamiento_general

In [38]:
genera_profiling_general(agua)

El dataframe tiene 17 columnas.
Hay 5 columnas que tienen NA's.


,Estadisticas,Resultado
0,Total de variables,17
1,Conteo de observaciones,71102
2,Total de celdas,1208734
3,Cantidad de variables numericas,11
4,Cantidad de variables de fecha,0
5,Cantidad de variables categóricas,0
6,Cantidad de variables de texto,6
7,Valores faltantes,26318
8,Porcentaje de valores faltantes,2.2%
9,Renglones duplicados,0


In [39]:
cuenta_nulos_por_columnas(agua)

El dataframe tiene 17 columnas.
Hay 5 columnas que tienen NA's.


,Missing Values,% del Total
consumo_total_mixto,8327,11.7
consumo_prom_mixto,8327,11.7
consumo_prom_dom,4820,6.8
consumo_total_dom,4820,6.8
Geo Shape,24,0.0


In [40]:
def cuenta_nulos_por_renglones(df):
    """
    Función que cuenta la cantidad de valores nulos por cada renglón, para valorar si es posible o no realizar
    imputaciones o se tendrían que tirar las columnas o renglones correspondientes.
    """
    df_aux = df.copy()
    valores_nulos_totales = sum(df_aux.apply(lambda x: sum(x.isnull().values), axis=1) > 0)
    print("Existen un total de: ", valores_nulos_totales, "renglones con al menos un valor nulo\n")
    numero_de_lineas = len(df_aux)
    porcentaje_de_lineas_con_nulos = valores_nulos_totales / numero_de_lineas
    texto = "Representan el {:.2%} del total de renglones.". \
        format(porcentaje_de_lineas_con_nulos)
    print(texto)
    return valores_nulos_totales

In [41]:
cuenta_nulos_por_renglones(agua)

Existen un total de:  8908 renglones con al menos un valor nulo

Representan el 12.53% del total de renglones.


8908

In [60]:
def cuenta_nulos_por_renglones_tabla(df):
    arreglo_nulos=[]
    arreglo_renglones=[]
    for i in range(len(df.index)):
        string = "Nan in row "+ str(i)
        valor = df.iloc[i].isnull().sum()
        arreglo_renglones.append(string)
        arreglo_nulos.append(valor)
        #print("Nan in row ", i, " : ", df.iloc[i].isnull().sum())
    sum([True for idx, row in df.iterrows() if any(row.isnull())])
    list(df.index[df.isnull().sum(axis=1) > 0])
    data={'renglon':arreglo_renglones,'valores_nulos':arreglo_nulos}
    tabla_valores_nulos_ordenada = pd.DataFrame(data=data)
    tabla_valores_nulos_ordenada_solonulos = pd.DataFrame(tabla_valores_nulos_ordenada)
    tabla_valores_nulos_ordenada_solonulos = tabla_valores_nulos_ordenada_solonulos.loc[tabla_valores_nulos_ordenada_solonulos["valores_nulos"] > 0]
    tabla_valores_nulos_ordenada_solonulos=tabla_valores_nulos_ordenada_solonulos.sort_values('valores_nulos', ascending = False)
    
    return tabla_valores_nulos_ordenada_solonulos.head(10)

In [61]:
cuenta_nulos_por_renglones_tabla(agua)

,renglon,valores
54555,Nan in row 54555,5
13306,Nan in row 13306,4
24048,Nan in row 24048,4
24061,Nan in row 24061,4
52584,Nan in row 52584,4
24052,Nan in row 24052,4
55696,Nan in row 55696,4
24050,Nan in row 24050,4
24049,Nan in row 24049,4
52583,Nan in row 52583,4


## Perfilamiento por variable

In [67]:
# Dividimos variables por tipo de datos
vars_type = agua.dtypes
vars_type = pd.DataFrame(vars_type, columns = ['tipo'])
vars_type['variable']=vars_type.index
vars_type

,tipo,variable
Geo Point,object,Geo Point
Geo Shape,object,Geo Shape
consumo_total_mixto,float64,consumo_total_mixto
anio,int64,anio
nomgeo,object,nomgeo
consumo_prom_dom,float64,consumo_prom_dom
consumo_total_dom,float64,consumo_total_dom
alcaldia,object,alcaldia
colonia,object,colonia
consumo_prom_mixto,float64,consumo_prom_mixto


In [71]:
# variables numericas
variables_int = vars_type.loc[vars_type["tipo"] == "int64"]
variables_float = vars_type.loc[vars_type["tipo"] == "float64"]
variables_numericas = variables_int.append(variables_float, ignore_index=True)
lista_numericas = list(variables_numericas['variable'])
lista_numericas

['anio',
 'bimestre',
 'gid',
 'consumo_total_mixto',
 'consumo_prom_dom',
 'consumo_total_dom',
 'consumo_prom_mixto',
 'consumo_total',
 'consumo_prom',
 'consumo_prom_no_dom',
 'consumo_total_no_dom']

In [73]:
# variables fecha
variables_date = vars_type.loc[vars_type["tipo"] == "Date"]
lista_date = list(variables_date['variable'])
lista_date

[]

In [74]:
# variables categoricas
variables_category = vars_type.loc[vars_type["tipo"] == "category"]
lista_category = list(variables_category['variable'])
lista_category

[]

In [75]:
# variables texto
variables_texto = vars_type.loc[vars_type["tipo"] == "object"]
lista_texto = list(variables_texto['variable'])
lista_texto

['Geo Point', 'Geo Shape', 'nomgeo', 'alcaldia', 'colonia', 'indice_des']

* Tipo de dato: float, integer
* Número de observaciones
* Mean
* Desviación estándar
* Cuartiles: 25%, 50%, 75%
* Valor máximo
* Valor mínimo
* Número de observaciones únicos
* Top 5 observaciones repetidas
* Número de observaciones con valores faltantes
* ¿Hay redondeos?

In [209]:
def genera_profiling_de_numericos(df):
    # Obtenemos los estadísticos de la columna si es numérica
    lista_perfilamiento_numerico = ['tipo','numero de observaciones', 'media', 'desviacion estándar',
                                    'cuartil 25%','cuartil 50%','cuartil 75%','minimo','maximo',
                                    'numero de observaciones unicas','top5 repetidos']
    datos_dataframe_profiling_numericas = {'metrica':lista_perfilamiento_numerico}
    dataframe_profiling_numericas = pd.DataFrame(data=datos_dataframe_profilin_numericas)
    for col in lista_numericas:
        # tipo de dato
        vars_type_num = pd.DataFrame(vars_type) 
        #vars_type_num
        df_tipo = pd.DataFrame(data=vars_type_num.loc[vars_type_num["variable"] == col])
        tipo_dato=df_tipo['tipo'][0]
        #print(tipo_dato)
    
        # Obtenemos las métricas relevantes
        descr_col = agua[col].describe()
        descr_col = pd.DataFrame(descr_col)
        descr_col['metrica']=descr_col.index
        descr_col.columns=['valor','metrica']
    
        # número de observaciones
        medida = 'count'
        metrica = descr_col.loc[descr_col["metrica"] == medida]
        num_observaciones_num = metrica['valor'][0]
        #print(num_observaciones_num)
    
        # media
        medida = 'mean'
        metrica = descr_col.loc[descr_col["metrica"] == medida]
        media_obs_num = metrica['valor'][0]
        media_obs_num = media_obs_num.round(2)
        #print(media_obs_num)
    
        # desviacion estándar
        medida = 'std'
        metrica = descr_col.loc[descr_col["metrica"] == medida]
        sd_obs_num = metrica['valor'][0]
        sd_obs_num = sd_obs_num.round(2)
        #print(sd_obs_num)
    
        # cuartil 25
        medida = '25%'
        metrica = descr_col.loc[descr_col["metrica"] == medida]
        cuant_25_obs_num = metrica['valor'][0]
        cuant_25_obs_num = cuant_25_obs_num.round(2)
        #print(cuant_25_obs_num)
    
        # cuartil 50
        medida = '50%'
        metrica = descr_col.loc[descr_col["metrica"] == medida]
        cuant_50_obs_num = metrica['valor'][0]
        cuant_50_obs_num = cuant_50_obs_num.round(2)
        #print(cuant_50_obs_num)
        #cuant_50_obs_num = agua.quantile(q=0.25)
        #print(cuant_50_obs_num)
    
        # cuartil 75
        medida = '75%'
        metrica = descr_col.loc[descr_col["metrica"] == medida]
        cuant_75_obs_num = metrica['valor'][0]
        cuant_75_obs_num = cuant_75_obs_num.round(2)
        #print(cuant_75_obs_num)
        #cuant_75_obs_num = agua.quantile(q=0.25)
        #print(cuant_75_obs_num)
    
        # minimo
        medida = 'min'
        metrica = descr_col.loc[descr_col["metrica"] == medida]
        minimo_obs_num = metrica['valor'][0]
        minimo_obs_num = minimo_obs_num.round(2)
        #print(minimo_obs_num)
    
        # maximo
        medida = 'max'
        metrica = descr_col.loc[descr_col["metrica"] == medida]
        maximo_obs_num = metrica['valor'][0]
        maximo_obs_num = maximo_obs_num.round(2)
        #print(maximo_obs_num)
    
        # numero de observaciones unicas
        num_obs_unicas_obs_num = agua[col].nunique()
        #print(num_obs_unicas_obs_num)
    
        # top 5 observaciones repetidas
        df_resultado = df[col].value_counts(dropna=True)
        df_resultado = pd.DataFrame(df_resultado)
        df_resultado.columns=['conteo_top_5']
        df_resultado=df_resultado.sort_values('conteo_top_5', ascending = False)
    
        top5 = df_resultado.head(5)
        #print(top5)
    
        # Número de observaciones con valores faltantes
        obs_faltantes_obs_num = agua[col].isna().sum()
        #print(obs_faltantes_obs_num)
    
        datos_variable = [tipo_dato,num_observaciones_num,media_obs_num,sd_obs_num, 
                          cuant_25_obs_num, cuant_50_obs_num,cuant_75_obs_num,minimo_obs_num,
                          maximo_obs_num,num_obs_unicas_obs_num,top5]
        dataframe_profiling_numericas[col]=datos_variable
    return dataframe_profiling_numericas

In [210]:
pd.set_option('display.max_colwidth', -1)
genera_profiling_de_numericos(agua)

/home/rafaelortega/.pyenv/versions/itam_intro_to_ds/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,metrica,anio,bimestre,gid,consumo_total_mixto,consumo_prom_dom,consumo_total_dom,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,consumo_total_no_dom
0,tipo,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64
1,numero de observaciones,71102,71102,71102,62775,66282,66282,62775,71102,71102,71102,71102
2,media,2019,2.01,35551.5,174.36,29.13,1186.26,50.64,1695.85,111.22,126.76,436.06
3,desviacion estándar,0,0.81,20525.5,312.66,64.57,2771.04,130.41,3555.7,1069.95,1095.82,2126.15
4,cuartil 25%,2019,1,17776.2,0,18.69,161.64,0,340.95,23.01,6.28,10.98
5,cuartil 50%,2019,2,35551.5,79.94,26.41,604.18,33.45,896.18,31.69,19.28,54.06
6,cuartil 75%,2019,3,53326.8,233.32,36.25,1261.45,61.22,1808.9,45.48,54.19,230.43
7,minimo,2019,1,1,0,0,0,0,0,0,0,0
8,maximo,2019,3,71102,23404.4,7796.41,95060.7,11702.2,119727,89691.8,89691.8,119727
9,numero de observaciones unicas,1,3,71102,24339,52060,47051,31911,56015,62214,37440,27336


In [191]:
# df_cuant_25 = pd.DataFrame(agua.quantile(q=0.25))
# df_cuant_25['columna']=df_cuant_25.index
# df_cuant_25

In [192]:
# descr_col = agua['anio'].describe()
# descr_col = pd.DataFrame(descr_col)
# descr_col['metrica']=descr_col.index
# descr_col.columns=['valor','metrica']
# #descr_col.loc[vars_type_num["metrica"] == 'count']
# descr_col

In [193]:
# metrica = descr_col.loc[descr_col["metrica"] == 'count']
# conteo = metrica['valor'][0]
# print(conteo)

In [194]:
# pd.DataFrame(agua.quantile(q=0.50))

In [195]:
#pd.DataFrame(agua.quantile(q=0.75))

In [196]:
# top 5 de observaciones repetidas
#df_resultado = agua['bimestre'].value_counts(dropna=True)

In [197]:
#df_resultado

In [198]:
#df_resultado = pd.DataFrame(df_resultado)

In [199]:
#df_resultado.head(5)

In [200]:
# def conteo_nulos_en_columna(df):
#    for i in range(len(df.index)):
#        print("Nan in row ", i, " : ", df.iloc[i].isnull().sum())
#    sum([True for idx, row in df.iterrows() if any(row.isnull())])
#    list(df.index[df.isnull().sum(axis=1) > 0])
#    tabla_valores_nulos_ordenada = 0
#    return tabla_valores_nulos_ordenada

In [201]:
#conteo_nulos_en_columna(agua['bimestre'])

In [202]:
#agua['consumo_total_mixto'].isna().sum()

In [217]:
# top 5 observaciones repetidas
df_resultado = agua['bimestre'].value_counts(dropna=True)
df_resultado = pd.DataFrame(df_resultado)
df_resultado.columns=['conteo_top_5']
df_resultado=df_resultado.sort_values('conteo_top_5', ascending = False)
    
top5 = df_resultado.head(5)

In [218]:
top5

,conteo_top_5
2,23942
3,23822
1,23338


In [219]:
#top5['conteo_top_5']=str(top5['conteo_top_5'])+'\n'

In [220]:
top5

,conteo_top_5
2,23942
3,23822
1,23338
